## ruby_experiment/src/Experiment.rb

In [1]:
module Element
  def initialize(factors = [])
    @factors = factors
  end
  attr_reader :factors
  #---
  def to_s() @factors.map(&:to_s).join end
  def show() self.to_s end
  def length() @factors.length end
  def ==(anElement) self.show == anElement.show end
end; nil

In [2]:
class Letter
  include Element
  #---
  def initialize(char='1')
    raise ArgumentError, "You can use alphabet and '1'" \
      unless char[0] =~ /[1a-zA-Z]/
    super([char])
    @char = char[0]
  end
  attr_reader :char
  #---
  def =~(a_Letter)
    raise ArgumentError unless a_Letter.is_a? Letter
    @char.downcase == a_Letter.char.downcase
  end
  def ==(a_Letter)
    raise ArgumentError unless a_Letter.is_a? Letter
    (self =~ a_Letter) && (self.inverse? == a_Letter.inverse?)
  end
  def ===(a_Letter)
    raise(ArgumentError) unless a_Letter.is_a? Letter
    self.object_id == a_Letter.object_id
  end
  def <=>(a_Letter)
    raise(ArgumentError) unless a_Letter.is_a? Letter
    self.show <=> another.show
  end
  #---
  def inverse
    if @char == '1' then
      self.dup
    else
      inv_char = (self.inverse?) ? @char.downcase : @char.upcase
      self.class.new(inv_char)
    end
  end
  def inverse?() (@char == '1') ? nil : (@char == @char.upcase) end
  def inverse_of?(a_Letter)
    (@char == '1') ? (a_Letter.char == '1') : (self =~ a_Letter) && !(self == a_Letter)
  end
end; nil

In [3]:
ltr1,ltr2 = Letter.new('a'), Letter.new('e')
[ltr1.show,ltr1.inverse?,ltr2.show,ltr2.inverse.show]

["a", false, "e", "E"]

In [51]:
class Word
  include Element
  #---
  # Word class consists of a (nested) array of Letter objects.
  #---
  def initialize(*args)
    args.flatten!
    @factors = []
    if args.empty?
        @factors << Letter.new('1')
    else
      args.each do |arg|
        @factors << case arg
                    when Letter then
                      arg
                    when String then
                      subarr = []
                      arg.each_char{|c| subarr << Letter.new(c) if c =~ /[1a-zA-Z]/}
                      if subarr.empty?
                        Letter.new('1')
                       elsif subarr.length == 1
                         subarr[0]
                      else
                        subarr
                      end
                    else raise ArgumentError 
                    end
      end
      @factors = @factors[0] if (@factors.length == 1 and @factors[0].is_a?(Array))
    end
#     super(factors)
  end
  #---
  def pop() @factors.pop end
  def letter_at(int) @factors[int] end
  def [](int) self.letter_at(int) end
  #---
  def flatten
    flat_factors = @factors.flatten
    self.class.new(flat_factors)
  end
  def flatten!() @factors.flatten!; self end
  #---
  def show() show_parens end 
  def show_parens()
    @factors.map{|f| (f.length == 1) ? f.show : "(#{self.class.new(f).show_parens})"}.join
  end
  #---
  def contract_once
#     self.flatten!
    rtn = [@factors[0]]
    @factors.each_cons(2) do |f1, f2|
      (f1 =~ f2 and f1 != f2) ? rtn.pop : rtn << f2
    end
    @factors = (rtn.empty?) ? [Letter.new('1')] : rtn 
    return self
  end
  def contract
    self.flatten!
    size_diff = 1
    while (size_diff > 0 and @factors.size > 1)
      size_diff = @factors.size - self.contract_once.factors.size
    end
    #
    return self
  end
  #---
  def *(anElement)
    raise ArgumentError unless anElement.is_a? Element
    return self.class.new(self.show, anElement.show)
  end
end; nil

In [5]:
id = Word.new; id.show

"1"

In [41]:
w = Word.new(Letter.new('a'),'B','Cd')
# w.flatten!
w.show

"aB(Cd)"

In [60]:
v = Word.new('kvm')
pd = w*v
p pd.factors, pd.show
p pd.contract.factors; nil

[[#<Letter:0x000055f10e9af1f0 @factors=["a"], @char="a">, #<Letter:0x000055f10e9af060 @factors=["B"], @char="B">, #<Letter:0x000055f10e9aee30 @factors=["C"], @char="C">, #<Letter:0x000055f10e9aebb0 @factors=["d"], @char="d">], [#<Letter:0x000055f10e9ae6d8 @factors=["k"], @char="k">, #<Letter:0x000055f10e9ae570 @factors=["v"], @char="v">, #<Letter:0x000055f10e9ae408 @factors=["m"], @char="m">]]
"(aBCd)(kvm)"
[#<Letter:0x000055f10e9af1f0 @factors=["a"], @char="a">, #<Letter:0x000055f10e9af060 @factors=["B"], @char="B">, #<Letter:0x000055f10e9aee30 @factors=["C"], @char="C">, #<Letter:0x000055f10e9aebb0 @factors=["d"], @char="d">, #<Letter:0x000055f10e9ae6d8 @factors=["k"], @char="k">, #<Letter:0x000055f10e9ae570 @factors=["v"], @char="v">, #<Letter:0x000055f10e9ae408 @factors=["m"], @char="m">]


In [7]:
myw = Word.new('baCcabCcAaBc'); 
"#{myw.show} --contract--> #{myw.contract.show}"
# myw.factors

"baCcabCcAaBc --contract--> baac"

In [37]:
module Group extend self
  Identity = Word.new('1')
  #---
  def product(element1,element2)
    raise ArgumentError unless [element1, element2].all?{|elm| elm.is_a? Element}
    return Word.new(element1.factors, element2.factors)
  end
end; nil 

<main>:1: warning: already initialized constant Group::Identity
<main>:1: warning: previous definition of Identity was here


In [38]:
e = Group::Identity
p e, e.class, e.show; nil

#<Word:0x000055f10e941b78 @factors=[#<Letter:0x000055f10e941998 @factors=["1"], @char="1">]>
Word
"1"


In [39]:
elem1, elem2 = Word.new('abc'), Word.new('def') # Word.new('abc'), Letter.new('d')

pdt = Group.product(elem1, elem2)
p pdt
pdt.show

#<Word:0x000055f10e9041d8 @factors=[#<Letter:0x000055f10e904f20 @factors=["a"], @char="a">, #<Letter:0x000055f10e904db8 @factors=["b"], @char="b">, #<Letter:0x000055f10e904c50 @factors=["c"], @char="c">, #<Letter:0x000055f10e904930 @factors=["d"], @char="d">, #<Letter:0x000055f10e9047c8 @factors=["e"], @char="e">, #<Letter:0x000055f10e904458 @factors=["f"], @char="f">]>


"abcdef"

In [17]:
  #---
  def product_with(another)
    if @char == '1' then
      another
    elsif another.to_s == '1' then
      self
    elsif self.inverse_of?(another) then
      Word::Identity
    else
      Word.new * self * another
    end
  end


:product_with